# CorrelAid machine-learning spring school 
## Section 5: Data Exploration 

Author: Sebastian Zezulka 

Session will take place on **Tuesday, 29.03.2022, 18h CEST**.

**to be distributed on Thursday, 24.03.2022**

During the next weeks, you will analyse the "Forest Cover Type Dataset" from [UCI Machine Learning](https://archive.ics.uci.edu/ml/datasets/covertype). In this section, you will learn how to set up a (primary) data exploration and to prepare your data for further (machine learning) analyses.
Doing this, you will also take a close look at the practical problem and the data. For data extrapolation, you will have to estimate descriptive statistics and build some visualisations of the data. Further, you will have to write a function to get a balanced dataset and build a training-test-split.

### 0. Set up

In [1]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

### 1. Load Data
**Task:** Download the CSV data from the given URL/repo and inspect a few rows from it. What size has the dataset?

In [2]:
print(os.getcwd())
  
# to change the working directory use
#os.chdir()

C:\Users\Sebastian\Documents\10_CorrelAid\02_Projekte\02_ml_winter_school\notebooks


In [3]:
# Read in csv data

# INSERT YOUR CODE HERE
path_name = 'data/covtype.csv'
forest_data = pd.read_csv(path_name)

# Inspect first few rows of the data

# INSERT YOUR CODE HERE
forest_data.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


## 2. Exploratory Analysis 

**Task:** Let's take a first look at the data we are going to analyse in the next weeks. Please make some notes for the workshop. We will predict the type of cover of a forest based on several features (variables) available. What, then, is the dependent variable in our data given the task? What was the original purpose to collect the data at hand? Do you have an intuition which features might be especially important and why?

**Answer:** 

**### delete before distribution ###**
'Cover_Type' is the dependent variable in our setting. 7 Categories are recorded: 
1 -- Spruce/Fir
2 -- Lodgepole Pine
3 -- Ponderosa Pine
4 -- Cottonwood/Willow
5 -- Aspen
6 -- Douglas-fir
7 -- Krummholz.
Data was collected to predict information on neighbouring lands of natural resource managers responsible for developing ecosystem management strategies. The study area includes four wilderness areas located in the Roosevelt National Forest of northern Colorado.

**Task:** Does the dataset contain missing values? Check how many categories are in 'Type of Cover'. How many observations do we have per category? Plot the number of observations per category. What do you see?

In [16]:
# check for missing values

# INSERT YOUR CODE HERE
forest_data.info()

AttributeError: 'Series' object has no attribute 'info'

In [18]:
# number of observations per cover type category 

# INSERT YOUR CODE HERE
forest_data.groupby('Cover_Type').count()

# easier even is
forest_data['Cover_Type'].value_counts()

2    283301
1    211840
3     35754
7     20510
6     17367
5      9493
4      2747
Name: Cover_Type, dtype: int64

In [ ]:
# bar plot of observations per cover type category 

# INSERT YOUR CODE HERE
list_soil_types = list(range(1,8))
num_obs = forest_data.groupby('Cover_Type').count().iloc[:,0]

plt.bar(list_soil_types, num_obs)
plt.xlabel('Category type of Cover')
plt.ylabel('# of observations')
plt.title('Number of observations per type of Cover category, original data')
plt.show();

**Answer:** 

**Task:** In the task above, we have seen that our data is unbalanced in the target variable `Type of Cover`. That is, there are different number of observations per category. This can be problematic for some machine learning algorithms, especially in classification, because much more information on classes 1 and 2 is available in training. This also can have implications for fairness concerns. In principle, we have three easy ways to deal with this problem: learn on the unbalanced data anyways; use *undersampling* by reducing the number of observations to the class with lowest observations; use *oversampling* by multiplying observations from categories with smaller number of observations or weight the loss function accordingly. You can get a more detailed overview [here](https://medium.com/strands-tech-corner/unbalanced-datasets-what-to-do-144e0552d9cd)

Here, we will apply undersampling only. Your task is to write a function that produces a balanced dataset by randomly sampling from the respective classes. For this, use the provided random number generator. 

In [ ]:
def undersampling(data, y_label: str, seed: int):
    '''
    Function that generates a balanced dataset from an unbalanced one
    input: 
        data : (N, D)-dataset
        y_label (string) : name of target variable
        seed (int) : seed for random number generator 
    
    output:
        data_balanced : dataset with balanced classes
    '''
    rng = np.random.default_rng(seed)
    
    # INSERT YOUR CODE HERE
    classes_y = sorted(data[y_label].unique())
    counts_y = data.groupby(y_label).count().iloc[:,0]
    
    data_balanced = []
    for i in classes_y:
        idx = rng.integers(low=0, high=counts_y[i], size=min(counts_y))
        data_i = pd.DataFrame(data[data[y_label] == i]).reset_index()
        data_i = data_i.loc[idx]
        data_balanced.append(data_i)
        
    data_balanced = pd.concat(data_balanced)
    
    return data_balanced

In [ ]:
forest_data_balanced = undersampling(forest_data, 'Cover_Type', 1)

In [ ]:
# test the undersampling function
list_soil_types = sorted(forest_data_balanced['Cover_Type'].unique())
num_obs = forest_data_balanced.groupby('Cover_Type').count().iloc[:,0]

plt.bar(list_soil_types, num_obs)
plt.xlabel('Category type of Cover')
plt.ylabel('# of observations')
plt.title('Number of observations per type of Cover category, balanced data')
plt.show();

**Task:** Two of our variables (`Wilderness_Area` and `Soil_Type`) are OneHot-encoded (also called dummy variables). That is, for every category we have one variable which indicates membership with a '1' and is '0' elsewhere. For the analysis, transform the variables of "Type Wilderness" and "Type Soil" into two categorical variables and drop the dummies. How many categories has each variable and what do they encode, respectivley? Is every category present in the data?

In [ ]:
# Display all variable names from dataset 
display(forest_data_balanced.columns)

# transform dummy variables into categorical ones and drop dummies

# INSERT YOUR CODE HERE
forest_data_balanced['Wilderness_Area_Type'] = (forest_data_balanced.iloc[:,11:15] == 1).idxmax(1)
forest_data_balanced['Soil_Type'] = (forest_data_balanced.iloc[:,15:55] == 1).idxmax(1)

# transform strings to numbers by, first, deleting text (use str.replace()) and, second, transforming variables to numeric
forest_data_balanced['Wilderness_Area_Type'] = forest_data_balanced['Wilderness_Area_Type'].str.replace('Wilderness_Area', '').astype('int64') # 'category'
forest_data_balanced['Soil_Type'] = forest_data_balanced['Soil_Type'].str.replace('Soil_Type', '').astype('int64') # 'category'

# drop dummy variables
forest_data_balanced.drop(['Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
       'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
       'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10',
       'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14',
       'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18',
       'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22',
       'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26',
       'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
       'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
       'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38',
       'Soil_Type39', 'Soil_Type40'], axis=1, inplace=True)

# check result
display(forest_data_balanced.head())
display(forest_data_balanced.info())

**Task:** Reorder the features in the dataset such that our feature variable `Cover_Type` is the last feature and also stored as Categorical data.

In [ ]:
# INSERT YOUR CODE HERE

forest_data_balanced['Cover_Type'] = forest_data_balanced['Cover_Type'].astype('int64') # 'category'

cols_at_end = ['Cover_Type']
forest_data_balanced = forest_data_balanced[[c for c in forest_data_balanced.columns.values if c not in cols_at_end] 
        + [c for c in cols_at_end if c in forest_data_balanced.columns.values]]

display(forest_data_balanced.head())

**Task:** Now, let us take a closer look at the data and some descriptive statistics. Please answer the following questions: What is the `Cover Type` with the highest mean distance to the next fire point? What is the standard deviation of hillshades at noon? What is measured by the features `Elevation`, `Aspect`, and `Slope`? What is their respective scale?

In [ ]:
# Baseline Descriptive Statistics
forest_data_describe = forest_data_balanced.describe()
display(forest_data_describe)

In [ ]:
# ordered distance to fire points by 'Cover Type' categories

# INSERT YOUR CODE HERE
forest_data_balanced.groupby('Cover_Type').mean()['Horizontal_Distance_To_Fire_Points'].sort_values(ascending = False)

In [ ]:
# standard deviation of the feature 'Hillshade at Noon'

# INSERT YOUR CODE HERE
forest_data_balanced['Hillshade_Noon'].std()

**Answer:**
- Cover Type 2 has largest mean of horizontal distance to next fire point: 2168.15 meters 
- std of Hillshade_Noon: 19.768
- Elevation: meters 
- Aspect:  degrees azimuth (https://en.wikipedia.org/wiki/Azimuth)
- Slope: degrees 

**Task:** In this last part of Section 2, we will look at some visualisations of the features. First, we will consider correlations between variables. 
Create a visual representation of the correlation matrix of the features. Do you find any patterns?

In [ ]:
# Correlation Matrix of Features

# INSERT YOUR CODE HERE
labels = forest_data_balanced.columns
x_ = range(len(labels))

cor = forest_data_balanced[labels].corr()

fig, ax = plt.subplots(1, 1, figsize=(15, 10))
im = plt.imshow(cor)
plt.xticks(ticks=x_, labels=labels, rotation=90)
plt.yticks(ticks=x_, labels=labels, rotation=0)
fig.colorbar(im, ax=ax)
plt.title('Correlation Matrix of numerical features')
plt.show();

**Answer:** Correlations between Hillshades at different points in time.

**Task:** Second, we will look at scatter plots between pairs of features. You can use the 'PairGrid'-function from [seaborn](https://seaborn.pydata.org/generated/seaborn.PairGrid.html) for this. On the diagonal, we plot the histograms of the respective feature. As before, can you make out any patterns in the data? 

In [ ]:
grid = sns.PairGrid(data = forest_data_balanced.iloc[:,:10], corner=True)

# INSERT YOUR CODE HERE

# Diagonal is a histogram
grid.map_diag(plt.hist, color = 'green', edgecolor = 'white')

# Bottom is scatter plot
grid.map_lower(sns.scatterplot, color = 'green', alpha = 0.1)

**Answer:** Something something one what one sees in these plots...

### 3. PCA and data-compression
This is a small exkurs! We won't discuss it (in detail) in the workshop, so feel free to skipp this part.

PCA - principal component analysis - is a simple (linear algebra) tool of unsupervised learning. In general, there are three motivations for using dimensionality reduction techniques: saving memory, finding "structure" in the data and finding differences between a known number of classes in the data. PCA can also be used as preprocessing step in a machine learning pipline. The core idea of PCA is to find a lower dimensional representation of the data such that the mean squared reconstruction error is minimised. One can show that this is achieved for projecting the (centered) data on the first m eigenvectors of the covariance (or correlation) matrix. Note that scaling of data changes the results. Also, it is hard (to impossible) to interpret results when features have not the same unit! 

You can find an intuitive explanation of PCA [here](https://stats.stackexchange.com/questions/2691/making-sense-of-principal-component-analysis-eigenvectors-eigenvalues). The same author, Dmitry Kobak, has an hour long introduction to PCA online [here](https://www.youtube.com/watch?v=xBf_LZ5ZgY4). Overall, his series of ten talks is a fantastic introduction to machine learning!

**Task:** Write a function that performs PCA on input data. For this, estimate the covariance matrix of the data and estimate its eigenvalues. Sort them in decreasing order and sort the corresponding eigenvectors accordingly. The eigenvalues dividided by the total variance give the fraction of variance "explained" by one principle component. Multiplying the eigenvectors with the data results in the principal components. 

In [ ]:
def PCA(data):
    '''
    Function that performs PCA on the input data
    
    input: (N, D)-shaped data
    output:
        fraction_variance_explained: (D,)-shaped array with the fraction of variance explained by the individual PCs
        principal_components: (D, D)-shaped array containing the principal components as columns
    '''
    # INSERT YOUR CODE HERE
    
    # covariate matrix of transposed data
    C = np.cov(data.T)
    # eigenvalues and -vectors of cov-matrix
    eigen_values, eigen_vectors = np.linalg.eigh(C)
    
    # sort eigenvalues and eigenvectors 
    idx = eigen_values.argsort()[::-1]
    eigen_values = eigen_values[idx]
    eigen_vectors = eigen_vectors[:,idx]

    # total variance is trace of C
    tot_var = np.trace(C)
    # Explained Variance by PC i 
    fraction_variance_explained = np.divide(eigen_values, tot_var)
    
    # principal components
    principal_components = eigen_vectors
        
    return fraction_variance_explained, principal_components

In [ ]:
# select only numerical features for PCA
pca_forest_data_balanced = forest_data_balanced.iloc[:,:10]

frac_var_exp, PC_forest_data = PCA(pca_forest_data_balanced)

**Task:** We first want to know how much variance the first principle components explain. For this, plot the variance explained of the `n`-th PC on the y-axis against the number of principal components on the x-axis. Second, plot the cumulative variance explained for the `n` PCs with highest variance explained (y-axis vs. the number of principal components (x-axis). 
From the latter plot you should be able to see how much PCs you need to keep to explain at least `x`% of the variance.

In [ ]:
plt.figure(figsize=(14,7))

plt.subplot(121)
# INSERT YOUR CODE HERE
plt.plot( range(len(frac_var_exp)), frac_var_exp, lw=2)

plt.title('variance explained by the n\'th PC')
plt.ylabel('share of variance explained in %')
plt.xlabel('n\'th PC')

plt.subplot(122)
# INSERT YOUR CODE HERE
plt.plot(range(len(frac_var_exp)), np.cumsum(frac_var_exp), lw=2)

plt.title('cummulative variance explained by PCs')
plt.ylabel('cum. share of variance explained in %')
plt.xlabel(r'n\'th PC')

plt.show()

**Task:** We now can also plot the first three PC scores, that is the data multiplied with the first `n` PCs against each other. Further, by colouring the datapoints by `Cover_Type` categories, we can see whether the first few principle components can separete the classes.

In [ ]:
# PCA scores of forest_data: multiply the data with the three eigenvectors that belong to the two largest eigenvalues

# INSERT YOUR CODE HERE
pca_scores =  pca_forest_data_balanced @ PC_forest_data[:,:3]

plt.scatter(pca_scores.iloc[:,0], pca_scores.iloc[:,1], c=forest_data_balanced['Cover_Type'])
plt.xlabel('PC1 score')
plt.ylabel('PC2 score')
plt.title('Biplot of principle component scores')
plt.show();
plt.scatter(pca_scores.iloc[:,1], pca_scores.iloc[:,2], c=forest_data_balanced['Cover_Type'])
plt.xlabel('PC2 score')
plt.ylabel('PC3 score')
plt.title('Biplot of principle component scores')
plt.show()

**Task:** Lastly, implement the PCA algorithm using the sklearn package and compare the results. 

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# INSERT YOUR CODE HERE
pca_sklearn = PCA(svd_solver='full')
pca_sklearn.fit(pca_forest_data_balanced)
    
pca_sklearn_frac_var_explained = pca_sklearn.explained_variance_ratio_
pca_sklearn_principal_components = pca_sklearn.components_.T

### 4. Preprocessing 

In this final section, we prepare the data for further (machine learning) analyses. For this, we split the data in a training and a test set and standardise it.

A core feature of machine learning is the usage of *strictily sparated* trainings and test data to check the generlisibility of a model. The background to this is so called "bias-variance trade-off". Intuitivley, we can have so many parameters that the model perfectly learns the data. That implies a zero loss on this data - after all, we have fitted a model perfectly on this data. The problem is that this approach will (in almost all cases) give really bad results when using this model on any new data. We say our model "overfitted". To control the generlisation performance, we train the model only on a subset of the available data (say, 80%) and test on the rest (20% then). Ultimatley, we are interested in a model that performs well on the test set, not the training. You will hear more about this in the next weeks. Only the test set result is a reasonable approximation of the performance of your model! 

When you have panel data and you assume they are indepent and identically distributed (iid), it is important to shuffle your data before you make the split to ensure that the data collection process does not "encode" information in your datasets. Note the difference to time-series data, where iid assumption is violated.

**Task:** Before, we have collapsed the OneHot encoded variables into one categorical. For the analysis, we have to re-transform the variables `Wilderness_Area_Type` and `Soil_Type ` into a OneHot encoding.

In [ ]:
# OneHotEncoding for `Wilderness Area'

# INSERT YOUR CODE HERE
ohe = OneHotEncoder(sparse=False)
wilderness_area_ohe = ohe.fit_transform(np.array(forest_data_balanced['Wilderness_Area_Type']).reshape(-1,1))
print('Categories for Wilderness Area: ' + str(ohe.categories_))

# OneHotEncoding for `Soil Type'

# INSERT YOUR CODE HERE
ohe2 = OneHotEncoder(sparse=False)
soil_type_ohe = ohe2.fit_transform(np.array(forest_data_balanced['Soil_Type']).reshape(-1,1))
print('Categories for Soil Type: ' + str(ohe2.categories_))

In [ ]:
# make pd.DataFrame with correct indices

# INSERT YOUR CODE HERE
idx = forest_data_balanced.index
wilderness_area_ohe = pd.DataFrame(wilderness_area_ohe, columns=['Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4'], index=idx)
soil_type_ohe = pd.DataFrame(soil_type_ohe, columns=[
        'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
        'Soil_Type6', 'Soil_Type7', 'Soil_Type9', 'Soil_Type10',
        'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14',
        'Soil_Type16', 'Soil_Type17', 'Soil_Type18',
        'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22',
        'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26',
        'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
        'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
        'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38',
        'Soil_Type39', 'Soil_Type40'], index=idx)

# merge OneHot encodings with the original dataset
forest_data_balanced = pd.concat([forest_data_balanced, wilderness_area_ohe, soil_type_ohe], axis=1)

**Task:** Now separate the dependent variable `y` from the rest of the data, `X`, and drop it. Then use the `train_test_split` function to create a test and trainings set, the test set should have size 20%.

In [ ]:
# separate the target feature in an array `y` and drop it from the rest of the data `X`

# INSERT YOUR CODE HERE
y = forest_data_balanced['Cover_Type']
X = forest_data_balanced.drop(['Cover_Type'] , axis = 1)
labels = X.columns

# test and training set split

# INSERT YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=42)
X_train.shape

In this last step, we standardise the continuous variables. That means one both centers the variable (i.e. subtracts the means) and divides by the standard deviation of the training data so that the variable has a standard deviations of 1. Importantly, the test data is standardised by using the statistics of the training data. If we standardise the whole dataset before splitting, we risk "information leakage" into the test set. Categorical (dummy) variables are left out and are later re-added. 

**Task:** Split of the categorical and numerical parts of both trainings and test data, use the instantiated function `StandardScale` to standardise both trainings and test data. Then, rejoin the datasets.

In [ ]:
# split of numerical variables for standardisation

# INSERT YOUR CODE HERE

label = X_train.columns
X_num_train = X_train[['Elevation', 'Aspect', 'Slope', 
                       'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
                       'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon',
                       'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']]
X_num_test = X_test[['Elevation', 'Aspect', 'Slope',
                       'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
                       'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon',
                       'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']]

# standardise numerical data
scaler = StandardScaler(with_mean=True, with_std=True)

# INSERT YOUR CODE HERE

scaler.fit(X_num_train.values)
X_num_train_scaled = scaler.transform(X_num_train.values)
X_num_test_scaled = scaler.transform(X_num_test.values)

# use column names and indices from above to transform scaled results in DataFrames

# INSERT YOUR CODE HERE

X_num_train_scaled = pd.DataFrame(X_num_train_scaled, index=X_num_train.index, columns=X_num_train.columns)
X_num_test_scaled = pd.DataFrame(X_num_test_scaled, index=X_num_test.index, columns=X_num_test.columns)

# rejoin the numerical and categorical data in two DataFrames: one for the scaled trainings and one for the scaled test data

# INSERT YOUR CODE HERE

train_scaled = pd.concat([X_num_train_scaled, X_train[[c for c in label if c not in X_num_train_scaled.columns.values]], y_train], axis=1)
test_scaled = pd.concat([X_num_test_scaled, X_test[[c for c in label if c not in X_num_test_scaled.columns.values]], y_test], axis=1)

**Task:** Save the two dataframes as csv files for future analyses.

In [ ]:
# save data as csv files

# INSERT YOUR CODE HERE

train_scaled.to_csv(r'Data\covertype_train_scaled.csv', index = False)
test_scaled.to_csv(r'Data\covertype_test_scaled.csv', index = False)

### Sources:
The PCA part is modelled after the course Machine Learning I by Dmitry Kobak, Uni Tübingen. 

Inspiration and idea for sns.PairGrid comes from Ceren Iyim (2020). [Predicting Forest Cover Types with the Machine Learning Workflow](https://towardsdatascience.com/predicting-forest-cover-types-with-the-machine-learning-workflow-1f6f049bf4df). Medium.

Thanks to Philipp Holzträger and Alexander Braun for helpful comments.